In [3]:
import tkinter as tk
from tkinter import messagebox
import pywhatkit as pwk
import psutil
import pyaudio
import speech_recognition as sr
import subprocess
import numpy as np
import cv2
from PIL import Image
from geopy.geocoders import Nominatim
from googlesearch import search
import webbrowser

def show_result(result_text):
    messagebox.showinfo("Result", result_text)

def listen_to_microphone():
    p = pyaudio.PyAudio()
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        show_result("Listening... Say something!")
        try:
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source, timeout=5)
            recognized_text = recognizer.recognize_google(audio)
            process_text(recognized_text.lower())
        except sr.WaitTimeoutError:
            show_result("No speech detected within the timeout.")
        except sr.UnknownValueError:
            show_result("Could not understand the audio.")
        except sr.RequestError as e:
            show_result("Error with the speech recognition service; {0}".format(e))
    p.terminate()

def process_text(text):
    trigger_word = "execute"
    if trigger_word in text:
        command = text.replace(trigger_word, "").strip()
        show_result(f"Executing command: {command}")
        subprocess.run(command, shell=True)
    else:
        show_result(f"You said: {text}")

def create_image():
    canvas = np.zeros((100, 100, 3), dtype=np.uint8)
    canvas[:, :] = [255, 255, 255]
    canvas[0:33] = [0, 0, 255]
    canvas[33:68] = [255, 255, 255]
    canvas[34:67, 34:67] = [255, 0, 0]
    canvas[68:100] = [0, 255, 0]
    cv2.imshow('hi', canvas)
    cv2.waitKey()
    cv2.destroyAllWindows()

def face_picture_crop():
    pic1 = cv2.imread("using.jpg")
    pic01 = pic1[10:400, 30:350]
    pic2 = cv2.imread("target.jpg")
    pic2[75:400, 80:350] = pic1[75:400, 80:350]
    cv2.imshow("faceCrop", pic2)
    cv2.waitKey()
    cv2.destroyAllWindows()

def paste_sunglasses():
    photo_path = "face.jpg"
    sunglasses_path = "sunglasses.png"
    output_path = "my_photo_with_sunglasses.jpg"

    face_width = 2000
    photo = Image.open(photo_path)
    sunglasses = Image.open(sunglasses_path)
    sunglasses_width = face_width // 1
    sunglasses = sunglasses.resize((sunglasses_width, sunglasses_width * sunglasses.height // sunglasses.width))
    paste_x = 1300
    paste_y = 1000
    photo.paste(sunglasses, (paste_x, paste_y), sunglasses)
    photo.save(output_path)
    show_result("Sunglasses applied to the image.")

def get_coordinates():
    geolocator = Nominatim(user_agent="GetLoc")
    location = geolocator.geocode("jaipur")
    if location:
        result_text = f"Address: {location.address}\nLatitude = {location.latitude}\nLongitude = {location.longitude}"
        show_result(result_text)
    else:
        show_result("Location not found.")

def google_search():
    search_query = input("Enter What You Want To Search for: ")
    top_5_results = search(search_query, num_results=5)
    result_text = f"Top 5 results for '{search_query}':\n"
    for i, result in enumerate(top_5_results, 1):
        result_text += f"{i}. {result}\n"
    show_result(result_text)

if __name__ == "_main_":
    root = tk.Tk()
    root.title("Menu")
    
    tk.Button(root, text="Find RAM Usage", command=lambda: show_result(f"RAM memory % used: {psutil.virtual_memory().percent}\nRAM Used (GB): {psutil.virtual_memory().used / 1000000000}")).pack()
    tk.Button(root, text="Send WhatsApp Message", command=lambda: pwk.sendwhatmsg_instantly('+918824610711', 'Hello', 10)).pack()
    tk.Button(root, text="Speech to Text", command=listen_to_microphone).pack()
    tk.Button(root, text="Create Image", command=create_image).pack()
    tk.Button(root, text="Face Picture Crop", command=face_picture_crop).pack()
    tk.Button(root, text="Apply Sunglasses to Image", command=paste_sunglasses).pack()
    tk.Button(root, text="Get GPS Coordinates", command=get_coordinates).pack()
    tk.Button(root, text="Google Search", command=google_search).pack()
    tk.Button(root, text="Exit", command=root.destroy).pack()

    root.mainloop()